### Given a fic_id, find its author_key, pseudos

In [1]:
import pandas as pd
import json
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import pprint
from datetime import datetime
import numpy as np
import requests
from bs4 import BeautifulSoup
import argparse
import time
import os
import csv
import sys
from unidecode import unidecode
from itertools import groupby
from operator import itemgetter

In [44]:
def find_author_of_ficid(fic_id):
    """para: fic_id(str)"""
    
    r1 = requests.get("https://archiveofourown.org/works/" + fic_id,
                        headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0'})
    soup1 = BeautifulSoup(r1.content, "html.parser")
    try:
        url_splits = soup1.find("a", rel="author")['href'].split('/')
        author_key = url_splits[2]
        pseud = url_splits[-1]
        return author_key, pseud
    except:
        print('Cannot find author for', fic_id)
        return None,None
    
    
    
    

In [45]:
find_author_of_ficid("17445209")

('falmarien', 'falmarien')

### Add author_key and pseud columns to ```TSNE_embed.csv```

In [46]:
tsne_file = 'sampledata/relationship_tsne_embed.csv'

In [ ]:
# input tsne_embed.csv
tsne_fin = open('sampledata/relationship_tsne_embed.csv')
rows = csv.reader(tsne_fin)
row0 = next(rows)
row0.append('author_key, pseud')
print(row0)

# output tsne_embed_wAuthor.csv
fout = open(tsne_file[:-4] + '_wAuthor.csv', mode = "w+")
foutwriter = csv.writer(fout)
foutwriter.writerow(row0)

lookup = {} 

for row in rows:
    fic_id = row[0]
    
    # check we have looked up this fic_id already
    if fic_id in lookup:
        author_key = lookup[fic_id][0]
        pseud = lookup[fic_id][1]
    else:
        author_key, pseud = find_author_of_ficid(fic_id)
        
    lookup[fic_id] = (author_key, pseud)
    foutwriter.writerow(row + [author_key, pseud])

tsne_fin.close()
fout.close()

['fic_id', 'character', 'relationship', 'additional_tags', 'category', 'd1', 'd2', 'label', 'author_key, pseud']
Cannot find author for 15959783
Cannot find author for 7796623
Cannot find author for 15781191
